<a href="https://colab.research.google.com/github/MrinmoyOpsAlpha/deterministic-windpower-forecasting/blob/main/Pelican_Optimzation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
device_list = tf.test.gpu_device_name()
device_list

'/device:GPU:0'

In [ ]:
df = pd.read_csv('/content/mycsv.csv')
df

,Date/Time,Date/Time (UTC),TOTAL WIND BASE SCHEDULE (FORECAST) IN BPA CONTROL AREA (MW; SCADA 187517,TOTAL WIND BASEPOINT (FORECAST) IN BPA CONTROL AREA (MW; SCADA 103349,TOTAL WIND GENERATION IN BPA CONTROL AREA (MW; SCADA 79687),TOTAL BPA CONTROL AREA LOAD (MW; SCADA 45583),TOTAL HYDRO GENERATION (MW; SCADA 79682),TOTAL FOSSIL/BIOMASS GENERATION (MW; SCADA 164377),TOTAL NUCLEAR GENERATION (MW; 70681),NET INTERCHANGE (MW; SCADA 45581),TOTAL SOLAR BASE SCHEDULE (FORECAST) IN BPA CONTROL AREA (MW; SCADA 187513),TOTAL SOLAR Basepoint (FORECAST) IN BPA CONTROL AREA (MW; SCADA 177165),TOTAL SOLAR GENERATION IN BPA CONTROL AREA (MW; SCADA 142383)
0,01-01-2022 00:00,01-01-2022 08:00,NaN,27.0,5.0,8320.0,10817.0,1096.0,1159.0,4757.0,NaN,0.0,0.0
1,01-01-2022 00:05,01-01-2022 08:05,NaN,24.0,5.0,8338.0,10960.0,1079.0,1165.0,4872.0,NaN,0.0,0.0
2,01-01-2022 00:10,01-01-2022 08:10,NaN,21.0,5.0,8337.0,11036.0,1056.0,1164.0,4925.0,NaN,0.0,0.0
3,01-01-2022 00:15,01-01-2022 08:15,NaN,21.0,6.0,8342.0,11084.0,1037.0,1165.0,4950.0,NaN,0.0,0.0
4,01-01-2022 00:20,01-01-2022 08:20,NaN,21.0,6.0,8354.0,11085.0,1032.0,1161.0,4930.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86383,10/27/22 23:35,10/28/22 06:35,579.0,701.0,670.0,5282.0,4361.0,1097.0,1152.0,1998.0,0.0,0.0,0.0
86384,10/27/22 23:40,10/28/22 06:40,579.0,686.0,678.0,5244.0,4332.0,1095.0,1149.0,2013.0,0.0,0.0,0.0
86385,10/27/22 23:45,10/28/22 06:45,579.0,678.0,680.0,5223.0,4366.0,1096.0,1149.0,2068.0,0.0,0.0,0.0
86386,10/27/22 23:50,10/28/22 06:50,579.0,681.0,687.0,5216.0,4351.0,1097.0,1149.0,2069.0,0.0,0.0,0.0


In [ ]:
df1 = df[['Date/Time', 'TOTAL WIND GENERATION  IN BPA CONTROL AREA (MW; SCADA 79687)']]
df1.rename(columns={'TOTAL WIND GENERATION  IN BPA CONTROL AREA (MW; SCADA 79687)': 'WindPower'}, inplace=True)
df1.set_index('Date/Time', inplace=True)
df1

<ipython-input-8-a6132729fce8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'TOTAL WIND GENERATION  IN BPA CONTROL AREA (MW; SCADA 79687)': 'WindPower'}, inplace=True)


,WindPower
Date/Time,
01-01-2022 00:00,5.0
01-01-2022 00:05,5.0
01-01-2022 00:10,5.0
01-01-2022 00:15,6.0
01-01-2022 00:20,6.0
...,...
10/27/22 23:35,670.0
10/27/22 23:40,678.0
10/27/22 23:45,680.0


In [ ]:
df1.isnull().sum()

WindPower    12
dtype: int64

In [ ]:
mean_wind_power = df1['WindPower'].mean()

df1['WindPower'].fillna(mean_wind_power, inplace=True)

<ipython-input-10-b328e906eddd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['WindPower'].fillna(mean_wind_power, inplace=True)


In [ ]:
df1.isnull().sum()

WindPower    0
dtype: int64

In [ ]:
df_new = df1.describe().transpose()
df_new

,count,mean,std,min,25%,50%,75%,max
WindPower,86388.0,732.712744,757.830476,0.0,81.0,423.0,1270.0,2641.0


In [ ]:
def df_to_X_y(df, window_size):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        # Extract a window of data with 'window_size' time steps
        window = df_as_np[i:i+window_size]

        # Append the window to X
        X.append(window)

        # Extract the label (next data point after the window)
        label = df_as_np[i+window_size]

        # Append the label to y
        y.append(label)

    # Convert X and y to NumPy arrays
    X = np.array(X)
    y = np.array(y)

    return X, y

In [ ]:
WINDOW_SIZE = 12
X1, y1 = df_to_X_y(df1, WINDOW_SIZE)
X1.shape, y1.shape

((86376, 12, 1), (86376, 1))

In [ ]:
length = len(df)

# Convert to integers
train_length = int(0.7 * length)
val_length = int(0.85 * length)

X_train1, y_train1 = X1[:train_length], y1[:train_length]
X_val1, y_val1 = X1[train_length:val_length], y1[train_length:val_length]
X_test1, y_test1 = X1[val_length:], y1[val_length:]

# Check the shapes
print(X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape)


(60471, 12, 1) (60471, 1) (12958, 12, 1) (12958, 1) (12947, 12, 1) (12947, 1)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Reshape and flatten X_train1, X_val1, X_test1
X_train1_reshaped = X_train1.reshape(-1, X_train1.shape[-1])
X_val1_reshaped = X_val1.reshape(-1, X_val1.shape[-1])
X_test1_reshaped = X_test1.reshape(-1, X_test1.shape[-1])

# Fit and transform the scalers on the training data
X_train1_normalized = scaler_X.fit_transform(X_train1_reshaped)
y_train1_normalized = scaler_y.fit_transform(y_train1.reshape(-1, 1))

# Reshape the normalized data back to the original shape
X_train1_normalized = X_train1_normalized.reshape(X_train1.shape)
y_train1_normalized = y_train1_normalized.reshape(y_train1.shape)

# Transform the validation and test data using the fitted scalers
X_val1_normalized = scaler_X.transform(X_val1_reshaped)
X_test1_normalized = scaler_X.transform(X_test1_reshaped)

# Reshape the normalized data back to the original shape for validation and test
X_val1_normalized = X_val1_normalized.reshape(X_val1.shape)
X_test1_normalized = X_test1_normalized.reshape(X_test1.shape)

y_val1_normalized = scaler_y.transform(y_val1.reshape(-1, 1))


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense
from tensorflow.keras.losses import MeanSquaredError

# Define the fitness function for the POA
def fitness_function(model, weights_vector, data, labels):
    # Convert the 1D weights vector to the list of weight matrices for the model
    weights = vector_to_weights(weights_vector, model)
    model.set_weights(weights)

    # Calculate the mean squared error loss
    predictions = model.predict(data, verbose=0)
    mse = MeanSquaredError()
    loss = mse(labels, predictions).numpy()
    return loss

# Helper function to convert a vector to a list of weights that matches the model's architecture
def vector_to_weights(vector, model):
    weights = []
    offset = 0
    for layer in model.layers:
        layer_weights = []
        for weight_shape in layer.get_weights():
            weight_size = np.product(weight_shape.shape)
            layer_weights.append(vector[offset:offset + weight_size].reshape(weight_shape.shape))
            offset += weight_size
        weights.append(layer_weights)
    return [layer for layer_set in weights for layer in layer_set]

# Define the POA optimization function
def poa(search_agents, max_iterations, lower_bound, upper_bound, fitness_func, model, data, labels):
    # Initialize the positions of pelicans (weights of the neural network)
    dimension = sum([weights.size for weights in model.get_weights()])
    X = np.random.uniform(low=lower_bound, high=upper_bound, size=(search_agents, dimension))

    # Evaluate initial fitness for the population
    fit = np.array([fitness_func(model, x, data, labels) for x in X])

    # Initialize the best solution found
    fbest = np.inf
    Xbest = None

    # POA loop
    for t in range(max_iterations):
        # Update the best candidate solution
        best_idx = np.argmin(fit)
        best = fit[best_idx]
        if best < fbest:
            fbest = best
            Xbest = X[best_idx]

        # Update location of food
        k = np.random.choice(search_agents)
        X_FOOD = X[k]
        F_FOOD = fit[k]

        # Update positions of each pelican
        for i in range(search_agents):
            # Exploration phase
            X_new = X[i] + np.random.rand() * (X_FOOD - X[i])
            X_new = np.clip(X_new, lower_bound, upper_bound)
            f_new = fitness_func(model, X_new, data, labels)
            if f_new <= fit[i]:
                X[i] = X_new
                fit[i] = f_new

            # Exploitation phase
            X_new = X[i] + 0.2 * (1 - t / max_iterations) * (2 * np.random.rand(dimension) - 1) * X[i]
            X_new = np.clip(X_new, lower_bound, upper_bound)
            f_new = fitness_func(model, X_new, data, labels)
            if f_new <= fit[i]:
                X[i] = X_new
                fit[i] = f_new

        # Logging and tracking best scores
        print(f"Iteration {t + 1}/{max_iterations}, Best Fitness: {fbest}")

    return fbest, Xbest

# Model definition (as provided)
model1 = Sequential()
model1.add(InputLayer((X1.shape[1], 1)))
model1.add(LSTM(64))
model1.add(Dense(8, activation='relu'))
model1.add(Dense(1, activation='linear'))

# Run POA
search_agents = 10
max_iterations = 30
lower_bound = -1
upper_bound = 1

best_loss, best_weights_vector = poa(search_agents, max_iterations, lower_bound, upper_bound,
                                     fitness_function, model1,
                                     X_train1_normalized, y_train1_normalized)


Iteration 1/30, Best Fitness: 0.36626991629600525
Iteration 2/30, Best Fitness: 0.04754215478897095
Iteration 3/30, Best Fitness: 0.04754215478897095
Iteration 4/30, Best Fitness: 0.04754215478897095
Iteration 5/30, Best Fitness: 0.04754215478897095
Iteration 6/30, Best Fitness: 0.04754215478897095
Iteration 7/30, Best Fitness: 0.04754215478897095
Iteration 8/30, Best Fitness: 0.0440058708190918
Iteration 9/30, Best Fitness: 0.024147672578692436
Iteration 10/30, Best Fitness: 0.024147672578692436
Iteration 11/30, Best Fitness: 0.024147672578692436
Iteration 12/30, Best Fitness: 0.024147672578692436
Iteration 13/30, Best Fitness: 0.024147672578692436
Iteration 14/30, Best Fitness: 0.023124216124415398
Iteration 15/30, Best Fitness: 0.023124216124415398
Iteration 16/30, Best Fitness: 0.023124216124415398
Iteration 17/30, Best Fitness: 0.022483505308628082
Iteration 18/30, Best Fitness: 0.020975405350327492
Iteration 19/30, Best Fitness: 0.020975405350327492
Iteration 20/30, Best Fitness:

In [ ]:
# Function to set the weights of the model from a vector
def set_model_weights(model, weights_vector):
    # Convert the 1D weights vector to the list of weight matrices for the model
    weights = vector_to_weights(weights_vector, model)
    model.set_weights(weights)

# Now you can call set_model_weights to set the best weights found by POA
set_model_weights(model1, best_weights_vector)

In [ ]:
# Make predictions with the best weights on the validation set
predictions = model1.predict(X_val1_normalized)

# Calculate the loss and other metrics on the validation set
validation_loss = MeanSquaredError()(y_val1_normalized, predictions).numpy()
# You can calculate other metrics like MAE, RMSE, MAPE here as well


405/405 [==============================] - 1s 2ms/step


In [ ]:
predictions

array([[0.26621586],
       [0.2654534 ],
       [0.2642957 ],
       ...,
       [0.10795477],
       [0.10858676],
       [0.10922906]], dtype=float32)

In [ ]:
from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError

y_true = X_val1_normalized
y_pred =  predictions

# Calculate MAE
mae_metric = MeanAbsoluteError()
mae_metric.update_state(y_true, y_pred)
mae_result = mae_metric.result().numpy()
print(f"Mean Absolute Error (MAE): {mae_result}")

mse = MeanSquaredError()
mse.update_state(y_true, y_pred)
mse_result = mse.result().numpy()
rmse_result = np.sqrt(mse_result)
print(f"Root Mean Squared Error (RMSE): {rmse_result}")


Mean Absolute Error (MAE): 0.063466377556324
Root Mean Squared Error (RMSE): 0.07790128141641617


In [ ]:
predictions = model1.predict(X_val1_normalized)
predictions1 = model1.predict(X_train1_normalized)
predictions2 = model1.predict(X_test1_normalized)

405/405 [==============================] - 1s 3ms/step


In [ ]:
import pandas as pd

# Convert the predictions to pandas DataFrames
df_val = pd.DataFrame(predictions, columns=['Validation Predictions'])
df_train = pd.DataFrame(predictions1, columns=['Training Predictions'])
df_test = pd.DataFrame(predictions2, columns=['Test Predictions'])

# Now let's save these DataFrames to Excel files
df_val.to_excel('/content/validation_predictions.xlsx')
df_train.to_excel('/content/training_predictions.xlsx')
df_test.to_excel('/content/test_predictions.xlsx')

# If you want to save all predictions in one Excel file with different sheets
with pd.ExcelWriter('/content/all_predictions.xlsx') as writer:
    df_train.to_excel(writer, sheet_name='Training Data')
    df_val.to_excel(writer, sheet_name='Validation Data')
    df_test.to_excel(writer, sheet_name='Test Data')


In [ ]:
from google.colab import files

files.download('/content/validation_predictions.xlsx')
files.download('/content/training_predictions.xlsx')
files.download('/content/test_predictions.xlsx')
# If you saved all predictions in one file:
files.download('/content/all_predictions.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>